In [1]:
import scanpy as sc

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [30]:
from tqdm import tqdm
# use a progress bar 

In [31]:
sample_folders = ["RB01_rep1", "RB01_rep2", "RB02_rep1", "RB02_rep2", "RB03_rep1", "RB03_rep2", "RB04", "RB05", "RB06", "RB07",]
# Create an empty list first
adata_list = []

In [32]:
# use a for-loop to read in all the files
for sample in tqdm(sample_folders): 
    pathname = "D:/Research raw data/"+sample
    print("Reading files at:" + pathname)
    adata=sc.read_10x_mtx("D:/Research raw data/"+sample)
    patient_id = sample[:4] # extract the patient info
    sample_id = sample # extract the sample info
    # we add the patient and sample information into our adata object, specifically, the .obs slot
    # .obs stands for observation i.e every single-cell is an observation
    adata.obs["patient_id"] = patient_id
    adata.obs["sample_id"] = sample_id
    # the function will complain that we need to run this
    adata.var_names_make_unique() # a var, stands for variable i.e. genes. so this function is to make the gene names unique
    # why? because multiple ensembl gene ids can map to the same gene e.g. spliced isoforms. sometimes we care about spliced isoforms and we want to keep them separate
    # so something like IL6, would be become IL6 and IL6-1
    adata_list.append(adata) # append to my adata_list above, iteratively

# finally, concatenate all the objects into a singlular one
adata = sc.concat(adata_list)
# print what the final object looks like
adata


  0%|          | 0/10 [00:00<?, ?it/s]

Reading files at:D:/Research raw data/RB01_rep1


 10%|█         | 1/10 [02:24<21:40, 144.49s/it]

Reading files at:D:/Research raw data/RB01_rep2


 20%|██        | 2/10 [05:10<20:58, 157.29s/it]

Reading files at:D:/Research raw data/RB02_rep1


 30%|███       | 3/10 [06:27<14:03, 120.48s/it]

Reading files at:D:/Research raw data/RB02_rep2


 40%|████      | 4/10 [07:04<08:45, 87.53s/it] 

Reading files at:D:/Research raw data/RB03_rep1


In [16]:
# check if location is directory.
import os

print(os.path.isdir("D:/Research raw data/RB01_rep1"))

# check if something is a file
print(os.path.isfile("D:/Research raw data/RB01_rep1/barcodes.tsv.gz"))


True
True


In [20]:
folder = "RB05"
folder[:5]

'RB05'

In [ ]:
# save the adata, so you don't have to keep reading in like this
adata.write_h5ad("D:/Research raw data/adata_rb.h5ad", compression="gzip")

In [ ]:
# in the future, you can open a new notebook and just do
# import scanpy as sc
# adata_rb = sc.read_h5ad("D:/Research raw data/adata_rb.h5ad")
# adata_rb

In [ ]:
# say if you have a control object read in the same as above:
# you can contenate like:
# adata_combined = sc.concat([adata_rb, adata_ctrl])
# print what the final object looks like 
# adata_combined